In [3]:
from datetime import datetime
import numpy as np
import pandas as pd
import googlemaps
from googleplaces import GooglePlaces, types, lang
import gmaps
import ipywidgets as widgets


### Configuring Google Places and GMaps

In [4]:
# Open Google Places API key file and read keys
# The file is assumed to contain only one single line with the API key
def read_file_content(filename):
    """ Given a filename,
        returns the content of this file
    """
    try:
        with open(filename, 'r') as f:
            return f.read()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
        
# Configure GooglePlaces and gmaps
PLACES_API_KEY = read_file_content("./API Keys/GOOGLE_PLACES_API_KEY.txt")
google_places = GooglePlaces(PLACES_API_KEY)
gmaps.configure(PLACES_API_KEY)


### Google Places nearby-searches centered around Stuttgart Hbf, Karlshöhe and Feuersee 

Compare search results for different search centers (or none), as well as different ranking parameters, to understand the difference between configurations and which one is most suitable for our purpose (building a restaurant database for a Speech assistant recommending nice restaurants for the user).

In [5]:
def google_places_nearby_search(lat, lng, rankby='prominence', radius=None):
    if (radius is None):
        query_result = google_places.nearby_search(
        lat_lng={'lat': lat, 'lng': lng}, 
        rankby=rankby,
        types=[types.TYPE_RESTAURANT] or [types.TYPE_CAFE] or [type.TYPE_BAR] or [type.TYPE_CASINO])
    else:
        query_result = google_places.nearby_search(
            lat_lng={'lat': lat, 'lng': lng}, 
            rankby=rankby,
            radius=radius,
            types=[types.TYPE_RESTAURANT] or [types.TYPE_CAFE] or [type.TYPE_BAR] or [type.TYPE_CASINO])
    return query_result

def build_place_geoloc_dict(search_results):
    result_dict = dict()
    for place in search_results.places:
        result_dict[place.name] = [place.geo_location] 
    return result_dict

def print_search_results(search_results):
    for place in search_results.places:
        print (place.name)
   

In [6]:
# Lat/lng Stuttgart Hbf
lat_hbf = 48.783333
lng_hbf = 9.183333

# Prominence-based search
search_result_hbf_prom_none = google_places_nearby_search(lat_hbf, lng_hbf, 'prominence')
search_result_hbf_prom_20000 = google_places_nearby_search(lat_hbf, lng_hbf, 'prominence', 20000)
search_result_hbf_prom_1500 = google_places_nearby_search(lat_hbf, lng_hbf, 'prominence', 1500)
search_result_hbf_prom_500 = google_places_nearby_search(lat_hbf, lng_hbf, 'prominence', 500)
# Distance-based search
search_result_hbf_dist_20000 = google_places_nearby_search(lat_hbf, lng_hbf, 'distance', 20000)
search_result_hbf_dist_1500 = google_places_nearby_search(lat_hbf, lng_hbf, 'distance', 1500)



In [7]:
# Lat/lng Karlshöhe
lat_karlsh = 48.767454
lng_karlsh = 9.164108

# Prominence-based search
search_result_karlsh_prom_none = google_places_nearby_search(lat_karlsh, lng_karlsh, 'prominence')
search_result_karlsh_prom_20000 = google_places_nearby_search(lat_karlsh, lng_karlsh, 'prominence', 20000)
search_result_karlsh_prom_1500 = google_places_nearby_search(lat_karlsh, lng_karlsh, 'prominence', 1500)
search_result_karlsh_prom_500 = google_places_nearby_search(lat_karlsh, lng_karlsh, 'prominence', 500)
# Distance-based search
search_result_karlsh_dist_20000 = google_places_nearby_search(lat_karlsh, lng_karlsh, 'distance', 20000)
search_result_karlsh_dist_1500 = google_places_nearby_search(lat_karlsh, lng_karlsh, 'distance', 1500)



In [8]:
# lat/lng Stuttgart, Feuersee
lat_feuer = 48.773350
lng_feuer = 9.165829

# Prominence-based search
search_result_feuer_prom_none = google_places_nearby_search(lat_feuer, lng_feuer, 'prominence')
search_result_feuer_prom_20000 = google_places_nearby_search(lat_feuer, lng_feuer, 'prominence', 20000)
search_result_feuer_prom_1500 = google_places_nearby_search(lat_feuer, lng_feuer, 'prominence', 1500)
search_result_feuer_prom_500 = google_places_nearby_search(lat_feuer, lng_feuer, 'prominence', 500)
# Distance-based search
search_result_feuer_dist_20000 = google_places_nearby_search(lat_feuer, lng_feuer, 'distance', 20000)
search_result_feuer_dist_1500 = google_places_nearby_search(lat_feuer, lng_feuer, 'distance', 1500)


In [9]:
# Print search results for prominence-based search without radius -> Top 20 in Stuttgart
print_search_results(search_result_hbf_prom_none)

Hotel Royal
Der Zauberlehrling
Alte Kanzlei
Oggi
Enchilada Stuttgart
5 Gourmet Restaurant & Bar
Valle
Sky Beach Stuttgart
BLOCK HOUSE Eberhardstraße
Arche Weinstube
VAPIANO Suttgart Bolzstrasse
Weinstube Fröhlich
Paulaner am alten Postplatz
Prince of India
Hotel-Restaurant Köhler
Ambiente Africa
MAREDO Steakhaus Stuttgart
BLOCK HOUSE Arnulf-Klett-Platz
CUBE Restaurant
Amici


### Visualizing the Search Results

In [10]:
# Add the search center and the search results to the figures layer 

def draw_map_with_legend(fig, center_lat, center_lng, result_dict, color, title):
    
    locations = []
    for geo_loc in list(result_dict.values()):
        lat = geo_loc[0]['lat']
        lng = geo_loc[0]['lng']
        locations.append([float(lat), float(lng)])
        
    # Build label list to label the markers with numbers: prefix each restaurant label with a number 
    # This number (= the first letter) will be the label shown on the respective marker
    chars = []
    for i in range(len(locations)):
        chars.append(chr(ord('a') + i).upper() + ' ')
    # Concatenate elementwise, use as mouseover text and legend
    prefixed_names = [i + j for i, j in zip(chars, list(result_dict.keys()))]
    
    # Split prefixed_names to form two columns as legend
    prefixed_names_1 = prefixed_names[0:10]
    prefixed_names_2 = prefixed_names[10:20]
 
    # Build  legend columns
    legend_1 = widgets.VBox(
        [widgets.HTML(legend_entry) for legend_entry in prefixed_names_1],
        layout=widgets.Layout(justify_content='flex-end', max_height='500px'))
    legend_2 = widgets.VBox(
        [widgets.HTML(legend_entry) for legend_entry in prefixed_names_2],
        layout=widgets.Layout(justify_content='flex-end', max_height='500px'))
       
    # Add layer for locations and center
    layer1 = gmaps.marker_layer(locations=locations, hover_text=prefixed_names, label=chars)
    layer2 = gmaps.symbol_layer([[center_lat, center_lng]], fill_color=color, stroke_color=color)
    fig.add_layer(layer1)
    fig.add_layer(layer2)
    
    title = widgets.VBox([widgets.HTML('<h4>' + title + '</h4>')])  
    return widgets.VBox([title, widgets.HBox([fig, legend_1, legend_2])])
        
 

In [11]:
# Build maps and figures for Stuttgart Hbf

fig1 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_hbf, lng_hbf], zoom_level=13)
map1 = draw_map_with_legend(fig1, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_hbf_prom_none), 'purple', 
                    'Stuttgart Hbf, prominence-based search') 

fig2 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_hbf, lng_hbf], zoom_level=13)
map2 = draw_map_with_legend(fig2, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_hbf_prom_20000), 'purple', 
                    'Stuttgart Hbf, prominence-based search with distance=20.0000') 


fig3 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_hbf, lng_hbf], zoom_level=13)
map3 = draw_map_with_legend(fig3, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_hbf_prom_1500), 'purple', 
                    'Stuttgart Hbf, prominence-based search with distance=1.500') 

fig4 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_hbf, lng_hbf], zoom_level=13)
map4 = draw_map_with_legend(fig4, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_hbf_prom_500), 'purple', 
                    'Stuttgart Hbf, prominence-based search with distance=500') 


fig5 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_hbf, lng_hbf], zoom_level=13)
map5 = draw_map_with_legend(fig5, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_hbf_dist_20000), 'purple', 
                    'Stuttgart Hbf, distance-based search with distance=20.000') 


fig6 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_hbf, lng_hbf], zoom_level=13)
map6 = draw_map_with_legend(fig6, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_hbf_dist_1500), 'purple', 
                    'Stuttgart Hbf, distance-based search with distance=1.500') 


In [12]:
# Build maps and figures for Stuttgart Karlshöhe

fig7 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_karlsh, lng_karlsh], zoom_level=13)
map7 = draw_map_with_legend(fig7, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_karlsh_prom_none), 'purple', 
                    'Stuttgart Karlshöhe, prominence-based search') 

fig8 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_karlsh, lng_karlsh], zoom_level=13)
map8 = draw_map_with_legend(fig8, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_karlsh_prom_20000), 'purple', 
                    'Stuttgart Karlshöhe, prominence-based search with distance=20.0000') 


fig9 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_karlsh, lng_karlsh], zoom_level=13)
map9 = draw_map_with_legend(fig9, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_karlsh_prom_1500), 'purple', 
                    'Stuttgart Karlshöhe, prominence-based search with distance=1.500') 

fig10 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_karlsh, lng_karlsh], zoom_level=13)
map10 = draw_map_with_legend(fig10, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_karlsh_prom_500), 'purple', 
                    'Stuttgart Karlshöhe, prominence-based search with distance=500') 


fig11 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_karlsh, lng_karlsh], zoom_level=13)
map11 = draw_map_with_legend(fig11, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_karlsh_dist_20000), 'purple', 
                    'Stuttgart Karlshöhe, distance-based search with distance=20.000') 


fig12 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_karlsh, lng_karlsh], zoom_level=13)
map12 = draw_map_with_legend(fig12, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_karlsh_dist_1500), 'purple', 
                    'Stuttgart Karlshöhe, distance-based search with distance=1.500') 


In [13]:
# Build maps and figures for Stuttgart Feuersee

fig13 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_feuer, lng_feuer], zoom_level=14)
map13 = draw_map_with_legend(fig13, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_feuer_prom_none), 'purple', 
                    'Stuttgart Feuersee, prominence-based search') 

fig14 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_feuer, lng_feuer], zoom_level=14)
map14 = draw_map_with_legend(fig14, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_feuer_prom_20000), 'purple', 
                    'Stuttgart Feuersee, prominence-based search with distance=20.0000') 


fig15 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_feuer, lng_feuer], zoom_level=13)
map15 = draw_map_with_legend(fig15, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_feuer_prom_1500), 'purple', 
                    'Stuttgart Feuersee, prominence-based search with distance=1.500') 

fig16 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_feuer, lng_feuer], zoom_level=13)
map16 = draw_map_with_legend(fig16, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_feuer_prom_500), 'purple', 
                    'Stuttgart Feuersee, prominence-based search with distance=500') 


fig17 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_feuer, lng_feuer], zoom_level=13)
map17 = draw_map_with_legend(fig17, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_feuer_dist_20000), 'purple', 
                    'Stuttgart Feuersee, distance-based search with distance=20.000') 


fig18 = gmaps.figure(layout={'width': '500px', 'height': '350px'}, center=[lat_feuer, lng_feuer], zoom_level=13)
map18 = draw_map_with_legend(fig18, lat_hbf, lng_hbf, build_place_geoloc_dict(search_result_feuer_dist_1500), 'purple', 
                    'Stuttgart Feuersee, distance-based search with distance=1.500') 


#### Stuttgart Hbf

In [14]:
widgets.VBox([map1, map2, map3, map4, map5, map6])

#### Stuttgart Karlshöhe

In [18]:
widgets.VBox([map7, map8, map9, map10, map11, map12])

#### Stuttgart Feuersee

In [19]:
widgets.VBox([map13, map14, map15, map16, map17, map18])